In [41]:
import serial
import csv
import time
from datetime import datetime

# ===============================
# 사용자 설정
# ===============================
SERIAL_PORT = 'COM4'    # ESP32 연결 포트
BAUD_RATE = 9600        # ESP32 시리얼 속도
CSV_FILENAME = 'air_quality_z.csv'


In [42]:
try:
    ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=1)
    time.sleep(2)  # ESP32 초기화 시간
    print(f"[INFO] Serial connected: {SERIAL_PORT} @ {BAUD_RATE}bps")
except Exception as e:
    print(f"[ERROR] Serial connection failed: {e}")
    ser = None


[INFO] Serial connected: COM4 @ 9600bps


In [43]:
try:
    csvfile = open(CSV_FILENAME, 'w', newline='')
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(['timestamp', 'ppm', 'temp', 'humidity', 'state'])
    csvfile.flush()
    print(f"[INFO] CSV file created: {CSV_FILENAME}")
except Exception as e:
    print("[ERROR] CSV file creation failed:", e)
    csvfile = None


[INFO] CSV file created: air_quality_z.csv


In [44]:
try:
    while True:
        line = ser.readline().decode('utf-8', errors='ignore').strip()
        if not line:
            continue

        print("받은 데이터:", line)

        # 시리얼 잡음이나 헤더 무시
        if "PPM" in line and "TEMP" in line:
            continue  # 헤더 라인 무시

        # 숫자 3개가 아니면 스킵
        parts = line.split(',')
        if len(parts) != 3:
            print("[WARN] Invalid data format, skipping")
            continue


        parts = line.split(',')
        if len(parts) != 3:
            print("[WARN] Invalid data format, skipping")
            continue

        try:
            ratio, temp, hum = map(float, parts)
        except ValueError:
            print("[WARN] Conversion failed, skipping line")
            continue
        
        

        ppm = ratio
        if ppm <= 400:
            state = 'GOOD'
        elif ppm <= 1000:
            state = 'NORMAL'
        elif ppm <= 2000:
            state = 'BAD'
        else:
            state = 'DANGER'

        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        # CSV 기록
        csvwriter.writerow([timestamp, round(ppm,1), temp, hum, state])
        csvfile.flush()

        # 콘솔 출력
        print(f"{timestamp} | PPM={round(ppm,1)} | T={temp}C | H={hum}% | State={state}")

except KeyboardInterrupt:
    print("\n[INFO] 종료 중...")

finally:
    if csvfile and not csvfile.closed:
        csvfile.close()
    if ser and ser.is_open:
        ser.close()
    print("[INFO] CSV 파일 닫음, 시리얼 포트 종료")


받은 데이터: 9P>B.g!j!Ln%,(̦b<1!PPM,TEMP,HUMIDITY
받은 데이터: 56.4,26.8,36
2026-01-17 03:51:29 | PPM=56.4 | T=26.8C | H=36.0% | State=GOOD
받은 데이터: 54.1,26.8,35
2026-01-17 03:51:29 | PPM=54.1 | T=26.8C | H=35.0% | State=GOOD
받은 데이터: 54.1,26.8,35
2026-01-17 03:51:30 | PPM=54.1 | T=26.8C | H=35.0% | State=GOOD
받은 데이터: 54.1,26.8,35
2026-01-17 03:51:31 | PPM=54.1 | T=26.8C | H=35.0% | State=GOOD
받은 데이터: 54.1,26.8,35
2026-01-17 03:51:32 | PPM=54.1 | T=26.8C | H=35.0% | State=GOOD
받은 데이터: 50.8,26.8,35
2026-01-17 03:51:33 | PPM=50.8 | T=26.8C | H=35.0% | State=GOOD
받은 데이터: 54.1,26.8,35
2026-01-17 03:51:34 | PPM=54.1 | T=26.8C | H=35.0% | State=GOOD
받은 데이터: 58.9,26.8,35
2026-01-17 03:51:35 | PPM=58.9 | T=26.8C | H=35.0% | State=GOOD
받은 데이터: 61.5,26.8,35
2026-01-17 03:51:36 | PPM=61.5 | T=26.8C | H=35.0% | State=GOOD
받은 데이터: 61.5,26.8,35
2026-01-17 03:51:37 | PPM=61.5 | T=26.8C | H=35.0% | State=GOOD
받은 데이터: 58.9,26.8,35
2026-01-17 03:51:38 | PPM=58.9 | T=26.8C | H=35.0% | State=GOOD
받은 데이터: 58.9,26.8,3